In [1]:
import logging
import warnings
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

warnings.filterwarnings("ignore")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [2]:
from src.pyaecal.trigger import Trigger
fn = Trigger()

In [3]:
fn.add_channel('n_Eng','n')
fn.add_channel('T_Eng_curr','tq')
fn.set_trigger_pre('n')
data=fn.get_data('./data/DEMO_AT.mdf')
data

INFO:root:./data/DEMO_AT.mdf: 62188 rows


offset,n,tq
f32,f32,f32
0.0,850.0,8.0
0.01,850.0,8.0
0.02,850.0,8.0
0.03,850.826843,8.0
0.04,851.0,8.0
…,…,…
621.830688,850.0,-3.25
621.840637,850.0,-3.25
621.850647,850.0,-3.25


In [4]:
import polars as pl
import logging
from src.pyaecal.trigger import Trigger  # Assumes trigger.py is available

# Set up logging
logging.basicConfig(level=logging.DEBUG)

class EngineTriggerDemo(Trigger):

    def __init__(self):
        super().__init__()
        # Add channels with renaming
        self.add_channel('n_Eng', rename='n')
        self.add_channel('T_Eng_curr', rename='tq')
        # Set triggers
        self.set_trigger_pre('n_above_851', up=True)  # Rising edge when n > 900
        self.set_trigger_post('n_above_851', up=False)  # Falling edge when n < 890

    def evaluate(self, data):
        data=data.with_columns(pl.when(pl.col("n") >= 851).then(1).otherwise(0).alias("n_above_851"))
        print(data)
        return super().evaluate(data)

In [5]:
demo=EngineTriggerDemo()
demo.add_file('./data/DEMO_AT.mdf')
demo.process()

INFO:root:./data/DEMO_AT.mdf: 62188 rows
shape: (62_188, 4)
┌────────────┬────────────┬───────┬─────────────┐
│ offset     ┆ n          ┆ tq    ┆ n_above_851 │
│ ---        ┆ ---        ┆ ---   ┆ ---         │
│ f32        ┆ f32        ┆ f32   ┆ i32         │
╞════════════╪════════════╪═══════╪═════════════╡
│ 0.0        ┆ 850.0      ┆ 8.0   ┆ 0           │
│ 0.01       ┆ 850.0      ┆ 8.0   ┆ 0           │
│ 0.02       ┆ 850.0      ┆ 8.0   ┆ 0           │
│ 0.03       ┆ 850.826843 ┆ 8.0   ┆ 0           │
│ 0.04       ┆ 851.0      ┆ 8.0   ┆ 1           │
│ …          ┆ …          ┆ …     ┆ …           │
│ 621.830688 ┆ 850.0      ┆ -3.25 ┆ 0           │
│ 621.840637 ┆ 850.0      ┆ -3.25 ┆ 0           │
│ 621.850647 ┆ 850.0      ┆ -3.25 ┆ 0           │
│ 621.860657 ┆ 850.0      ┆ -3.25 ┆ 0           │
│ 621.870667 ┆ 850.0      ┆ -3.25 ┆ 0           │
└────────────┴────────────┴───────┴─────────────┘


{'DEMO_AT': {'pre_triggers': shape: (74, 4)
  ┌────────────┬────────────┬───────────┬─────────────┐
  │ offset     ┆ n          ┆ tq        ┆ n_above_851 │
  │ ---        ┆ ---        ┆ ---       ┆ ---         │
  │ f32        ┆ f32        ┆ f32       ┆ i32         │
  ╞════════════╪════════════╪═══════════╪═════════════╡
  │ 0.04       ┆ 851.0      ┆ 8.0       ┆ 1           │
  │ 1.230001   ┆ 851.0      ┆ 8.000757  ┆ 1           │
  │ 2.140002   ┆ 851.0      ┆ 8.245496  ┆ 1           │
  │ 3.120003   ┆ 851.0      ┆ 8.0       ┆ 1           │
  │ 4.630005   ┆ 851.956787 ┆ 31.996243 ┆ 1           │
  │ …          ┆ …          ┆ …         ┆ …           │
  │ 619.680664 ┆ 851.0      ┆ -2.75     ┆ 1           │
  │ 620.360657 ┆ 851.0      ┆ -2.98975  ┆ 1           │
  │ 620.500671 ┆ 851.0      ┆ -3.0      ┆ 1           │
  │ 620.950684 ┆ 851.0      ┆ -3.0      ┆ 1           │
  │ 621.370667 ┆ 851.0      ┆ -3.238546 ┆ 1           │
  └────────────┴────────────┴───────────┴─────────────┘,
  '